# Setup

In [ ]:
!pip install scipy==1.6.3
!pip install scikit_optimize==0.8.1
!pip install scikit_learn==0.24.2
! git clone https://github.com/zykhoo/predicting_hamiltonian_dynamics.git

     |████████████████████████████████| 27.4 MB 1.4 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 101 kB 7.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but y

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run all

In [ ]:
import numpy as np

experiment,sys,dim = "NN","oscillator",2
H = lambda x: 1/2*x[1]**2+ 1/2*x[0]**2
f1 = lambda x: x[1]
f2 = lambda x: -x[0]
spacedim = [(-1.,1.),(-1.,1.)]
fvector = lambda x: np.asarray([f1(x), f2(x)])
h= 0.1
x0, H0 = 0.,0.
initialcon = [64, 128, 256, 512, 1024]
LR=0.001
diagdist = np.sum(np.square(np.asarray([spacedim[0][0], spacedim[0][1]]), np.asarray([spacedim[1][0], spacedim[1][1]])))
epsilon = 0.1

from predicting_hamiltonian_dynamics import groundtruth_2dim
import numpy as np 

"""# Test dataset creation"""

from tqdm import tqdm
import time 

evaluation_length_long = 50
evaluation_length_one = 1

xxlong,yylong = np.linspace(spacedim[0][0], spacedim[0][1], 5), np.linspace(spacedim[1][0], spacedim[1][1], 5)
xlong,ylong = np.meshgrid(xxlong,yylong)
xxshort,yyshort = np.linspace(spacedim[0][0], spacedim[0][1], 20), np.linspace(spacedim[1][0], spacedim[1][1], 20)
xshort,yshort = np.meshgrid(xxshort,yyshort)

trajectories_groundtruth_start_long = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xlong),np.ravel(ylong)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_long,n_h = 1),0)
  trajectories_groundtruth_start_long = np.vstack((trajectories_groundtruth_start_long, results_200))
trajectories_groundtruth_start_long.shape # output no., start and final, p and q, full traj

trajectories_groundtruth_start_short = np.expand_dims(groundtruth_2dim.classicTrajectory(np.asarray([[0.4],[0.]]),f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
for i in tqdm(np.expand_dims(np.c_[np.ravel(xshort),np.ravel(yshort)],2)):
  results_200 = np.expand_dims(groundtruth_2dim.classicTrajectory(i,f1,f2,h = 0.1,N=evaluation_length_one,n_h = 1),0)
  trajectories_groundtruth_start_short = np.vstack((trajectories_groundtruth_start_short, results_200))
trajectories_groundtruth_start_short.shape # output no., start and final, p and q, full traj

within_array = groundtruth_2dim.get_within_array(trajectories_groundtruth_start_long, spacedim)

100%|██████████| 400/400 [00:00<00:00, 5085.02it/s]


In [ ]:
path = '/content/drive/MyDrive/SSI/Baseline v2/synthetic systems (upload)/'

In [ ]:
from predicting_hamiltonian_dynamics.models import NN_2dim
from predicting_hamiltonian_dynamics.models import GP_2dim
from predicting_hamiltonian_dynamics.models import PINN_2dim
from predicting_hamiltonian_dynamics.models import PIGP_2dim


for i in range(20):
  seed = i
  np.random.seed(seed=seed)
  for ini in initialcon: 

    start, final = groundtruth_2dim.CreateTrainingDataTrajClassicIntRandom(1,ini,spacedim,h,f1,f2,seed = seed,n_h = 1)

    delta = start.copy()
    delta[0,:] = f1(start)
    delta[1,:] = f2(start)

    """# NN"""

    from predicting_hamiltonian_dynamics.models import NN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


    wholemat, evalmat = NN_2dim.data_preprocessing(start, delta, device)

    import torch.optim as optim
    import time 

    net = NN_2dim.Net(dim,16,dim)
    starttime = time.time() 
    net = NN_2dim.train(net, wholemat, evalmat, optimizer=optim.Adam(net.parameters(), lr=LR), batchsize=10, iter=1600, )
    traintime = time.time()-starttime


    from tqdm import tqdm
    from predicting_hamiltonian_dynamics.models import NN_2dim
    from predicting_hamiltonian_dynamics import metrics


    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = NN_2dim.compute_metrics_NN(net, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Oscillator_random.txt', 'a')
    file_object.write('NN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# GP"""

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF
    from sklearn import preprocessing

    output = delta
    scaler = preprocessing.MinMaxScaler()
    input = scaler.fit_transform(start.transpose())
    kernel = 1 * RBF(length_scale=2., length_scale_bounds="fixed")
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, optimizer = None, random_state = seed)
    starttime = time.time() 
    gaussian_process.fit(input, output.transpose())
    traintime = time.time()-starttime
    gaussian_process.predict(scaler.transform([[0.7,0.8]]))

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = GP_2dim.compute_metrics_GP(gaussian_process, scaler, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Oscillator_random.txt', 'a')
    file_object.write('GP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


    """# PINN"""

    from predicting_hamiltonian_dynamics.models import PINN_2dim
    import torch

    if torch.cuda.is_available():
      device=torch.device('cuda')
    else:
      device=torch.device('cpu')
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    wholemat, evalmat = PINN_2dim.data_preprocessing(start, delta,device)
    net=PINN_2dim.Net(dim,16,1)
    starttime = time.time()
    net,avg_vallosses,avg_lossli,avg_f1li,avg_f2li,avg_f3li,avg_f4li=PINN_2dim.train(net,bs=10,num_epoch=5000,initial_conditions=initialcon,device=device,
                                                                                    wholemat=wholemat,evalmat=evalmat,x0=x0,H0=H0,dim=dim,LR=LR,patience=500,c1=1,c2=1,c3=1,c4=1)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PINN_2dim.compute_metrics_PINN(net, device, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)
    
    file_object = open(path + 'Oscillator_random.txt', 'a')
    file_object.write('PINN, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()

    """# PIGP"""

    from predicting_hamiltonian_dynamics.models import PIGP_2dim
    GP = PIGP_2dim.BertalanGP()
    starttime = time.time()
    GP.train(start,delta,h)
    traintime = time.time()-starttime

    MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield, _, _ = PIGP_2dim.compute_metrics_PIGP(GP, h, diagdist, xshort, yshort, xlong, ylong, evaluation_length_long, within_array, trajectories_groundtruth_start_long, trajectories_groundtruth_start_short, fvector)

    file_object = open(path + 'Oscillator_random.txt', 'a')
    file_object.write('PIGP, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s \n' %(ini, seed, traintime, MSE_long, time_long, MSE_long_naive, time_long_naive, MSE_within, time_within, MSE_within_naive, time_within_naive, MSE_onestep, time_onestep, MSE_vectorfield, time_vectorfield))
    file_object.close()


training loss tensor(0.7595, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.5834, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.1235, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.1503, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0025, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0018, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0013, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0004, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0004, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation loss tensor(0.0002, dtype=torch.float64, grad_fn=<MeanBackward0>)
training loss tensor(0.0002, dtype=torch.float64, grad_fn=<MeanBackward0>)
validation lo

  0%|          | 0/50 [00:00<?, ?it/s]/content/predicting_hamiltonian_dynamics/models/NN_2dim.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fstage = lambda stg: h * torch.squeeze(net(torch.tensor([q+stg,p]).float().transpose(1,0)),0).detach().numpy().transpose()[:dim]
100%|██████████| 50/50 [00:00<00:00, 141.89it/s]

100%|██████████| 50/50 [00:00<00:00, 1080.38it/s]

100%|██████████| 50/50 [00:00<00:00, 270.90it/s]

100%|██████████| 50/50 [00:00<00:00, 263.84it/s]

100%|██████████| 50/50 [00:00<00:00, 376.28it/s]

100%|██████████| 50/50 [00:00<00:00, 4080.46it/s]

100%|██████████| 6/6 [00:00<00:00, 229.48it/s]

100%|██████████| 50/50 [00:00<00:00, 408.15it/s]

100%|██████████| 50/50 [00:00<00:00, 4007.32it/s]

100%|██████████| 15/15 [00:00<00:00, 415.52it/s]

100%|███

 
epoch= 0  time= 0.020361900329589844  loss= 0.8911838134129842  val_loss= 1.7809081077575684  f1= 0.3356196495270219  f2= 0.43222211709513897  f3= 0.1217530866463979  f4= 0.001588938136116402 percent lr= 0.001
 
epoch= 10  time= 0.16633391380310059  loss= 0.6358209351698557  val_loss= 1.5994899272918701  f1= 0.3413738988641691  f2= 0.29333989432488  f3= 0.0001325433149759192  f4= 0.0009746112599596457 percent lr= 0.001
 
epoch= 20  time= 0.3121817111968994  loss= 0.660859098037084  val_loss= 1.4973470568656921  f1= 0.3018876650285483  f2= 0.35821971256023927  f3= 0.00017932891205418855  f4= 0.0005723776929198029 percent lr= 0.001
 
epoch= 30  time= 0.44788074493408203  loss= 0.5389461343487104  val_loss= 1.3410483598709106  f1= 0.2778568605919601  f2= 0.26010057596109953  f3= 0.00033844265271909535  f4= 0.0006502601517819877 percent lr= 0.001
 
epoch= 40  time= 0.5972871780395508  loss= 0.4666795755426089  val_loss= 1.1022111177444458  f1= 0.23535919843027564  f2= 0.22840389474791023

100%|██████████| 400/400 [00:01<00:00, 321.58it/s]


Start training with for 64 data points
Start computation of covariance matrix.
Covariance matrix of shape (64, 64)computed.
Start Cholesky decomposition of (64, 64) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (129, 64)


...

100%|██████████| 50/50 [00:00<00:00, 291.22it/s]

100%|██████████| 50/50 [00:00<00:00, 3022.53it/s]

100%|██████████| 50/50 [00:00<00:00, 281.12it/s]

100%|██████████| 50/50 [00:00<00:00, 583.49it/s]

100%|██████████| 50/50 [00:00<00:00, 389.06it/s]

100%|██████████| 50/50 [00:00<00:00, 2790.10it/s]

100%|██████████| 6/6 [00:00<00:00, 413.52it/s]

100%|██████████| 50/50 [00:00<00:00, 322.57it/s]

100%|██████████| 50/50 [00:00<00:00, 2389.70it/s]

100%|██████████| 15/15 [00:00<00:00, 261.84it/s]

100%|██████████| 50/50 [00:00<00:00, 654.71it/s]

100%|██████████| 50/50 [00:00<00:00, 350.81it/s]

100%|██████████| 50/50 [00:00<00:00, 365.08it/s]

100%|██████████| 50/50 [00:00<00:00, 191.17it/s]

100%|██████████| 50/50 [00:00<00:00, 2401.49it/s]

100%|██████████| 50/50 [00:00<00:00, 228.77it/s]

100%|██████████| 50/50 [00:00<00:00, 230.87it/s]

100%|██████████| 50/50 [00:00<00:00, 2352.12it/s]

100%|██████████| 50/50 [00:00<00:00, 253.45it/s]

100%|██████████| 50/50 [00:00<00:00, 362.25it/s

 
epoch= 0  time= 0.22553777694702148  loss= 0.6585240785668536  val_loss= 0.6697497889399529  f1= 0.3173731807633909  f2= 0.3272502475811594  f3= 0.012788415768510255  f4= 0.001112234246968123 percent lr= 0.001
 
epoch= 10  time= 2.6402974128723145  loss= 0.0015847180514289747  val_loss= 0.0016005882614990696  f1= 0.000705343140315861  f2= 0.0007282638965830452  f3= 6.354289859611665e-06  f4= 0.00014475672651915187 percent lr= 0.001
 
epoch= 20  time= 5.075030326843262  loss= 0.0011027317871084073  val_loss= 0.0012336110274191014  f1= 0.00045113821699680737  f2= 0.0005640193223629886  f3= 5.6885793334764835e-06  f4= 8.188566549834182e-05 percent lr= 0.001
 
epoch= 30  time= 7.471162557601929  loss= 0.0009036430629061126  val_loss= 0.0009885847874102182  f1= 0.00037074388053289685  f2= 0.0004624222587666266  f3= 6.7934790517165126e-06  f4= 6.36834438220686e-05 percent lr= 0.001
 
epoch= 40  time= 9.87427568435669  loss= 0.0006843550285472103  val_loss= 0.0007381109840935096  f1= 0.0002

100%|██████████| 400/400 [00:01<00:00, 336.10it/s]


Start training with for 1024 data points
Start computation of covariance matrix.
Covariance matrix of shape (1024, 1024)computed.
Start Cholesky decomposition of (1024, 1024) Matrix
Cholesky decomposition completed.
Create LHS of linear system for H at test points.
Creation of linear system completed.
Solve least square problem of dimension (2049, 1024)


100%|██████████| 400/400 [07:45<00:00,  1.16s/it]
